In [1]:
import snowflake.connector
import pandas as pd

from statsmodels.tsa.arima.model import ARIMA
import matplotlib.pyplot as plt

CONEXÃO

In [2]:
conn = snowflake.connector.connect(
    user='isadoramonteiro'
    , password='engTI.5441'
    , account='SGNVEPB-LM56145' 
    , warehouse='compute_wh'  
    , database='aw'    
    , schema='prd'        
)

In [3]:
cursor = conn.cursor()

In [ ]:
# Fechar o cursor e a conexão
cursor.close()
conn.close()

CARREGAR OS DADOS

In [4]:
query = """SELECT 
            NM_CLIENTE
            , NOME_PRODUTO
            , ANO
            , MES
            , DATA_REF
            , QUANTIDADE
            , REGIAO
        from agg_sales_store """

cursor.execute(query)
rows = cursor.fetchall()

columns = [desc[0] for desc in cursor.description]
df_cliente = pd.DataFrame(rows, columns=columns)

print(df_cliente.head(100))

           NM_CLIENTE        NOME_PRODUTO   ANO  MES    DATA_REF  QUANTIDADE  \
0   Extreme Toy Store        Front Brakes  2013   12  2013-12-01           1   
1   Extreme Toy Store   HL Bottom Bracket  2013   12  2013-12-01           2   
2   Extreme Toy Store   HL Bottom Bracket  2014    3  2014-03-01           1   
3   Extreme Toy Store   HL Mountain Pedal  2013    6  2013-06-01           2   
4   Extreme Toy Store         LL Crankset  2013    9  2013-09-01           1   
..                ...                 ...   ...  ...         ...         ...   
95  First Cycle Store  Road-250 Black, 48  2014    5  2014-05-01           3   
96  First Cycle Store  Road-250 Black, 52  2013    7  2013-07-01           1   
97  First Cycle Store  Road-250 Black, 52  2013   10  2013-10-01           1   
98  First Cycle Store  Road-250 Black, 52  2014    5  2014-05-01           4   
99  First Cycle Store  Road-250 Black, 58  2013    7  2013-07-01           1   

     REGIAO  
0   Alabama  
1   Alabama

PREVISÃO DE DEMANDA

In [7]:
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

def forecast_demand(df_cliente):
    results = []

    # Análise para cada produto e loja
    for NOME_PRODUTO in df_cliente['NOME_PRODUTO'].unique():
        for NM_CLIENTE in df_cliente['NM_CLIENTE'].unique():
            # Filtrar os dados de acordo com o produto e loja
            df_filtered = df_cliente[(df_cliente['NOME_PRODUTO'] == NOME_PRODUTO) & (df_cliente['NM_CLIENTE'] == NM_CLIENTE)]
            
            # Verificar se existem dados suficientes (não podemos modelar se houver menos de 5 observações)
            if len(df_filtered) > 5:
                df_filtered = df_filtered[['DATA_REF', 'QUANTIDADE']]
                
                # Agregar as quantidades por data (soma ou média)
                df_filtered = df_filtered.groupby('DATA_REF')['QUANTIDADE'].sum().reset_index()

                # Ordenar os dados por data para garantir que o índice seja crescente
                df_filtered['DATA_REF'] = pd.to_datetime(df_filtered['DATA_REF'])
                df_filtered = df_filtered.sort_values('DATA_REF')

                df_filtered.set_index('DATA_REF', inplace=True)
                
                # Definir a frequência do índice como mensal ('MS' para o início de cada mês)
                df_filtered = df_filtered.asfreq('MS', method='pad')

                model = ARIMA(df_filtered, order=(1, 0, 0))  # Ajustar os parâmetros conforme necessário
                model_fit = model.fit()
                forecast = model_fit.forecast(steps=3)
                
                result = {
                    'NOME_PRODUTO': NOME_PRODUTO,
                    'NM_CLIENTE': NM_CLIENTE,
                    'forecast_1': round(forecast.iloc[0], 1),  # Primeiro mês
                    'forecast_2': round(forecast.iloc[1], 1),  # Segundo mês
                    'forecast_3': round(forecast.iloc[2], 1)   # Terceiro mês
                }
                results.append(result)

    return results

results = forecast_demand(df_cliente)


C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'
C:\Users\user\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationar

In [8]:
df = pd.DataFrame(results)

# Exibindo o DataFrame
print(df.head(100))

               NOME_PRODUTO                NM_CLIENTE  forecast_1  forecast_2  \
0    Mountain-200 Black, 46  Professional Cycle Store         2.5         2.8   
1    Mountain-200 Black, 46       Real Sporting Goods         8.7         7.1   
2    Mountain-200 Black, 46          Many Bikes Store         2.5         2.8   
3    Mountain-200 Black, 46           Resale Services         5.3         4.7   
4    Mountain-200 Black, 46           Small Bike Shop         2.3         2.6   
..                      ...                       ...         ...         ...   
95  Mountain-200 Silver, 46      Initial Bike Company         4.1         4.2   
96  Mountain-200 Silver, 46     Immediate Repair Shop         6.7         6.4   
97  Mountain-200 Silver, 46    Registered Cycle Store         2.9         3.4   
98  Mountain-200 Silver, 46           Spare Parts Co.         3.0         2.9   
99  Mountain-200 Silver, 46    Noiseless Gear Company         3.1         3.2   

    forecast_3  
0         

In [9]:
df_cleaned = df[~df[['forecast_1', 'forecast_2', 'forecast_3' ]].eq(0).all(axis=1)]
print(df_cleaned)

                NOME_PRODUTO                      NM_CLIENTE  forecast_1  \
0     Mountain-200 Black, 46        Professional Cycle Store         2.5   
1     Mountain-200 Black, 46             Real Sporting Goods         8.7   
2     Mountain-200 Black, 46                Many Bikes Store         2.5   
3     Mountain-200 Black, 46                 Resale Services         5.3   
4     Mountain-200 Black, 46                 Small Bike Shop         2.3   
...                      ...                             ...         ...   
1726        Road-650 Red, 44        Advanced Bike Components         4.6   
1727        Road-650 Red, 44               Fitness Toy Store         7.2   
1728        Road-650 Red, 44         Totes & Baskets Company         6.7   
1729        Road-650 Red, 44  Sure & Reliable Sporting Goods         2.9   
1730        Road-650 Red, 44                  Good Bike Shop         2.2   

      forecast_2  forecast_3  
0            2.8         3.0  
1            7.1         

In [10]:
df_cleaned.to_csv('8-previsao_demanda.csv')